In [1]:
import tkinter as tk
from tkinter import ttk, messagebox
from datetime import datetime


In [2]:
#tariff data: contains rental rates for different vehicles types with different durations
tariffs = {
    "small_car": {"daily": 60, "weekly": 200, "monthly":1500, "driver": 20},
    "sedan_car": {"daily": 80, "weekly": 450, "monthly":2200, "driver": 25},
    "suv": {"daily": 100, "weekly": 600, "monthly":2500, "driver": 30},
    }

#hire package data: fixed base charges + extra time charges for exceeding time limits
hire_packages = {
    "airport_drop_package": {"base_charge": 60, "max_km": 70, "extra_km_rate": 5, 
    "max_hour": 3, "extra_time_charge": 20}, 
    "100km_day_package": {"base_charge": 80, "max_km": 100, "extra_km_rate": 5, 
    "max_hour": 10, "extra_time_charge": 15},
}

#additional night charges
night_charges = {"driver_overnight_charges": 60, "vehicle_night_charges": 20}


In [3]:
def calculate_rent():
    vehicle_type = vehicle_var.get()
    rented_date = rent_start_entry.get().strip()
    return_date = rent_end_entry.get().strip()
    with_driver = driver_var.get()
    
    # Validation: Ensure inputs are provided
    if not vehicle_type:
        messagebox.showerror("Error", "Please select a vehicle.")
        return
    
    if not rented_date or not return_date:
        messagebox.showerror("Error", "Please enter both rent start and end dates.")
        return

    # Validation: Ensure correct date format and logic
    try:
        start_date = datetime.strptime(rented_date, "%d-%m-%Y")
        end_date = datetime.strptime(return_date, "%d-%m-%Y")
        if end_date <= start_date:
            messagebox.showerror("Error", "Return date must be after start date.")
            return
    except ValueError:
        messagebox.showerror("Error", "Invalid date format. Use YYYY-MM-DD.")
        return
    
    # Calculate rental cost
    rent_data = tariffs.get(vehicle_type)
    days = (end_date - start_date).days
    weeks = days // 7
    extra_days = days % 7
    total_rent = (weeks * rent_data['weekly']) + (extra_days * rent_data['daily'])
    
    if with_driver:
        total_rent += days * rent_data['driver']
    
    result_label.config(text=f"Total Rent: ${total_rent:.2f}")


In [4]:
def calculate_day_hire():
    vehicle_type = vehicle_var.get()
    package_type = package_var.get()

    #validation: ensure selections are made
    if not vehicle_type or not package_type:
        messagebox.showerror("Error", "Please select both vehicle and package.")
        return
    
    #Validation: Ensure numeric inputs for KM readings
    try:
        start_km = int(start_km_entry.get().strip())
        end_km = int(end_km_entry.get().strip())

        if end_km <= start_km:
            messagebox.showerror("Error", "End KM must be greater than start KM")
            return
        
    except ValueError:
        messagebox.showerror("Error", "Please enter valid numeric values for KM readings")
        return
    
    package = hire_packages.get(package_type)
    total_km = end_km - start_km
    extra_km_charge = max(0, (total_km - package['max_km']) * package['extra_km_rate'])

    total_hire = package['base_charge'] + extra_km_charge
    result_label.config(text = f"Total Hire: ${total_hire:.2f}")

In [6]:
#initialise main window
root = tk.Tk()
root.geometry("350x410")
root.config(bg="lightblue")
root.title("Rent a Car")

#Variables to hold user inputs
vehicle_var = tk.StringVar()
driver_var = tk.BooleanVar()
package_var = tk.StringVar()

#Vehicle Selection
tk.Label(root, text="Choose the Vehicle you want to rent:").pack()
vehicle_dropdown = ttk.Combobox(root, textvariable=vehicle_var, values = list(tariffs.keys()))
vehicle_dropdown.pack()

#rent calculation inputs
tk.Label(root, text="Enter Renting Date in DD-MM-YYYY Format:").pack()
rent_start_entry = tk.Entry(root)
rent_start_entry.pack()

tk.Label(root, text ="Enter Return Date in DD-MM-YYYY Format:").pack()
rent_end_entry = tk.Entry(root)
rent_end_entry.pack()

tk.Checkbutton(root, text="Do You Need a Driver? (Extra Charges Will Be Applied)", variable =driver_var).pack()
tk.Button(root, text="Calculate Rent", command=calculate_rent, font=("Cooper Black", 14), bg="pink").pack(pady=5)

#Hire Calculation Inputs
tk.Label(root, text="Select Your Rental Package:").pack()
package_dropdown = ttk.Combobox(root, textvariable=package_var, values=list(hire_packages.keys())).pack()

tk.Label(root, text="Enter Your Starting KM Reading:").pack()
start_km_entry = tk.Entry(root)
start_km_entry.pack()

tk.Label(root, text ="Enter Your Ending KM Reading:").pack()
end_km_entry = tk.Entry(root)
end_km_entry.pack()

tk.Button(root, text="Calculate Hire", command=calculate_rent, font=("Cooper Black", 14), bg="pink").pack(pady=5)

#result display
result_label = tk.Label(root, text = "", font = ("Cooper Black", 14), fg ="green")
result_label.pack()

root.mainloop()